In [64]:
import json
import os
import pandas as pd

# Define path
file_path = r"C:\Users\kyram\OneDrive\School\Prepossesed Data.json"

# Confirm the file exists
if not os.path.exists(file_path):
    print(f"The file does not exist at the specified path: {file_path}")
else:
    # Load the JSON 
    with open(file_path, 'r') as file:
        preprocessed_data = json.load(file)

    # Print the keys
    print(preprocessed_data.keys())

    # Access the key
    if 'public' in preprocessed_data:
        public_data = preprocessed_data['public']
        # Convert the dictionary back to a DataFrame
        df = pd.DataFrame.from_dict(public_data)
        print(df)
    else:
        print("'public' key not found in the JSON file")


dict_keys(['public'])
       Age   AsAm  AmerInd  AvgDrinksPerWeek  Black  DrinkDaysPerWeek  \
0     83.0  False      0.0               0.0    0.0               0.0   
1     58.0  False      0.0              35.0    0.0               7.0   
2     40.0  False      0.0               0.0    0.0               0.0   
3     66.0  False      0.0               0.0    0.0               0.0   
4     39.0  False      0.0               0.0    1.0               0.0   
...    ...    ...      ...               ...    ...               ...   
6247  48.0  False      0.0               4.0    0.0               2.0   
6248  50.0  False      0.0               0.0    0.0               0.0   
6249  55.0  False      0.0               3.0    1.0               1.0   
6250  62.0  False      0.0              84.0    0.0               7.0   
6251  57.0  False      0.0               0.0    0.0               0.0   

      DrinksOneOccasion  DrinksPerDay  Education  EverHadCancer  ...  \
0                   0.0      

In [65]:
# Import the necessary libraries
import numpy as np
import random
import sys
import tensorflow 

from collections import Counter

from imblearn.over_sampling import RandomOverSampler

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [66]:
# Define X and y
X = df.copy().drop(['EverHadCancer', 'FamilyEverHadCancer2'], axis=1)

y = df['EverHadCancer']

In [67]:
# Encode y
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y)

# Impute X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


In [68]:
# Scale X
numerical_transformer = StandardScaler()

X_imputed = numerical_transformer.fit_transform(X_imputed)


In [69]:
# Split into train and test splits
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_numeric, test_size=0.2, random_state=42, stratify=y_numeric)

# Resample
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

In [70]:
# Check shapes and distributions
print("Shape:", df.shape)
print("Dataset distribution:", Counter(df))

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("y_test Dataset distribution:", Counter(y_test))

print("X__train_resampled Shape", X_train_resampled.shape)
print("y_train_resampled Shape", y_train_resampled.shape)
print("y_train_resampled Dataset distribution:", Counter(y_train_resampled))

Shape: (6252, 59)
Dataset distribution: Counter({'Age': 1, 'AsAm': 1, 'AmerInd': 1, 'AvgDrinksPerWeek': 1, 'Black': 1, 'DrinkDaysPerWeek': 1, 'DrinksOneOccasion': 1, 'DrinksPerDay': 1, 'Education': 1, 'EverHadCancer': 1, 'FamilyEverHadCancer2': 1, 'FreqGoProvider': 1, 'HealthInsurance2': 1, 'HeardHPVVaccine2': 1, 'HPVCauseCancer_Cervical': 1, 'Hisp': 1, 'IncomeRanges': 1, 'MedConditions_Diabetes': 1, 'MedConditions_HighBP': 1, 'MedConditions_HeartCondition': 1, 'MedConditions_LungDisease': 1, 'OthPacIsl': 1, 'Smoke100': 1, 'Sunburned_Active': 1, 'Sunburned_Other': 1, 'Sunburned_Rec': 1, 'Sunburned_Work': 1, 'TimesSunburned': 1, 'UsedECigEver': 1, 'WhenPapTest': 1, 'White': 1, 'BirthGender_Female': 1, 'BirthGender_Male': 1, 'DelayNeededCare_I did not need any medical care in the past 12 months': 1, 'DelayNeededCare_No, I received the medical care I felt I needed': 1, 'DelayNeededCare_Yes': 1, 'DocTalkLDCT_Dont know': 1, 'DocTalkLDCT_I have never heard of this test': 1, 'DocTalkLDCT_No':

In [71]:
# Set seed
random.seed(42)

# Define model
model_1 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(30, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_1.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_1, accuracy_1 = model_1.evaluate(X_test, y_test)
print(f'Loss: {loss_1}, Accuracy: {accuracy_1}')

# Predict y
y_pred_1 = model_1.predict(X_test)
y_pred_classes_1 = np.argmax(y_pred_1, axis=1)

# F1 score and confusion matrix
f1_1 = f1_score(y_test, y_pred_classes_1, average='weighted')
conf_matrix_1 = confusion_matrix(y_test, y_pred_classes_1)

print(f'F1 Score: {f1_1}')
print('Confusion Matrix:')
print(conf_matrix_1)


Epoch 1/100
215/215 [==============================] - 1s 1ms/step - loss: 0.5580 - accuracy: 0.6925 - val_loss: 0.7296 - val_accuracy: 0.6025
Epoch 2/100
215/215 [==============================] - 0s 916us/step - loss: 0.4692 - accuracy: 0.7604 - val_loss: 0.7081 - val_accuracy: 0.6398
Epoch 3/100
215/215 [==============================] - 0s 925us/step - loss: 0.4265 - accuracy: 0.7956 - val_loss: 0.5772 - val_accuracy: 0.7140
Epoch 4/100
215/215 [==============================] - 0s 944us/step - loss: 0.3883 - accuracy: 0.8244 - val_loss: 0.5365 - val_accuracy: 0.7554
Epoch 5/100
215/215 [==============================] - 0s 923us/step - loss: 0.3534 - accuracy: 0.8460 - val_loss: 0.4690 - val_accuracy: 0.8056
Epoch 6/100
215/215 [==============================] - 0s 932us/step - loss: 0.3216 - accuracy: 0.8644 - val_loss: 0.5335 - val_accuracy: 0.7630
Epoch 7/100
215/215 [==============================] - 0s 929us/step - loss: 0.2892 - accuracy: 0.8817 - val_loss: 0.4250 - val_accu

In [72]:
# Set seed
random.seed(42)

# Define model
model_2 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(30, activation='relu'), 
    Dense(15, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_2.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_2, accuracy_2 = model_2.evaluate(X_test, y_test)
print(f'Loss: {loss_2}, Accuracy: {accuracy_2}')

# Predict y
y_pred_2 = model_2.predict(X_test)
y_pred_classes_2 = np.argmax(y_pred_2, axis=1)

# F1 score and confusion matrix
f1_2 = f1_score(y_test, y_pred_classes_2, average='weighted')
conf_matrix_2 = confusion_matrix(y_test, y_pred_classes_2)

print(f'F1 Score: {f1_2}')
print('Confusion Matrix:')
print(conf_matrix_2)


Epoch 1/100
215/215 [==============================] - 2s 4ms/step - loss: 0.5572 - accuracy: 0.6802 - val_loss: 0.6293 - val_accuracy: 0.7595
Epoch 2/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4612 - accuracy: 0.7624 - val_loss: 0.6279 - val_accuracy: 0.7665
Epoch 3/100
215/215 [==============================] - 0s 2ms/step - loss: 0.4058 - accuracy: 0.8051 - val_loss: 0.4890 - val_accuracy: 0.8465
Epoch 4/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3582 - accuracy: 0.8391 - val_loss: 0.4563 - val_accuracy: 0.8383
Epoch 5/100
215/215 [==============================] - 0s 2ms/step - loss: 0.3094 - accuracy: 0.8677 - val_loss: 0.3778 - val_accuracy: 0.8879
Epoch 6/100
215/215 [==============================] - 0s 2ms/step - loss: 0.2711 - accuracy: 0.8903 - val_loss: 0.3821 - val_accuracy: 0.8611
Epoch 7/100
215/215 [==============================] - 1s 2ms/step - loss: 0.2403 - accuracy: 0.9052 - val_loss: 0.3318 - val_accuracy: 0.8984

In [73]:
# Set seed
random.seed(42)

# Define model
model_3 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(40, activation='relu'), 
    Dense(20, activation='relu'),
    Dense(10, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_3.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_3, accuracy_3 = model_3.evaluate(X_test, y_test)
print(f'Loss: {loss_3}, Accuracy: {accuracy_3}')

# Predict y
y_pred_3 = model_3.predict(X_test)
y_pred_classes_3 = np.argmax(y_pred_3, axis=1)

# F1 score and confusion matrix
f1_3 = f1_score(y_test, y_pred_classes_3, average='weighted')
conf_matrix_3 = confusion_matrix(y_test, y_pred_classes_3)

print(f'F1 Score: {f1_3}')
print('Confusion Matrix:')
print(conf_matrix_3)


Epoch 1/100
215/215 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.6950 - val_loss: 0.7483 - val_accuracy: 0.6509
Epoch 2/100
215/215 [==============================] - 0s 876us/step - loss: 0.4494 - accuracy: 0.7782 - val_loss: 0.6385 - val_accuracy: 0.6912
Epoch 3/100
215/215 [==============================] - 0s 894us/step - loss: 0.3884 - accuracy: 0.8187 - val_loss: 0.6374 - val_accuracy: 0.6772
Epoch 4/100
215/215 [==============================] - 0s 994us/step - loss: 0.3422 - accuracy: 0.8441 - val_loss: 0.4666 - val_accuracy: 0.8079
Epoch 5/100
215/215 [==============================] - 0s 922us/step - loss: 0.2904 - accuracy: 0.8762 - val_loss: 0.4042 - val_accuracy: 0.8313
Epoch 6/100
215/215 [==============================] - 0s 964us/step - loss: 0.2473 - accuracy: 0.9013 - val_loss: 0.2804 - val_accuracy: 0.9043
Epoch 7/100
215/215 [==============================] - 0s 1ms/step - loss: 0.2136 - accuracy: 0.9159 - val_loss: 0.2810 - val_accura

In [74]:
# Set seed
random.seed(42)

# Define model
model_4 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(40, activation='relu'), 
    Dense(30, activation='relu'), 
    Dense(20, activation='relu'),
    Dense(10, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_4.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_4, accuracy_4 = model_4.evaluate(X_test, y_test)
print(f'Loss: {loss_4}, Accuracy: {accuracy_4}')

# Predict y 
y_pred_4 = model_4.predict(X_test)
y_pred_classes_4 = np.argmax(y_pred_4, axis=1)

# F1 score and confusion matrix
f1_4 = f1_score(y_test, y_pred_classes_4, average='weighted')
conf_matrix_4 = confusion_matrix(y_test, y_pred_classes_4)

print(f'F1 Score: {f1_4}')
print('Confusion Matrix:')
print(conf_matrix_4)


Epoch 1/100
215/215 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7023 - val_loss: 0.7257 - val_accuracy: 0.6532
Epoch 2/100
215/215 [==============================] - 0s 996us/step - loss: 0.4557 - accuracy: 0.7740 - val_loss: 0.5520 - val_accuracy: 0.7834
Epoch 3/100
215/215 [==============================] - 0s 879us/step - loss: 0.4000 - accuracy: 0.8163 - val_loss: 0.5066 - val_accuracy: 0.8074
Epoch 4/100
215/215 [==============================] - 0s 882us/step - loss: 0.3478 - accuracy: 0.8482 - val_loss: 0.4409 - val_accuracy: 0.8383
Epoch 5/100
215/215 [==============================] - 0s 867us/step - loss: 0.2981 - accuracy: 0.8730 - val_loss: 0.5409 - val_accuracy: 0.7572
Epoch 6/100
215/215 [==============================] - 0s 929us/step - loss: 0.2385 - accuracy: 0.9058 - val_loss: 0.3445 - val_accuracy: 0.8803
Epoch 7/100
215/215 [==============================] - 0s 939us/step - loss: 0.2042 - accuracy: 0.9207 - val_loss: 0.3632 - val_accu

In [75]:
# Compare the differnet versions of the model
print('Model_1: 1 layered model')
print(f'Loss: {loss_1}, Accuracy: {accuracy_1}')
print(f'F1 Score: {f1_1}')
print('Confusion Matrix:')
print(conf_matrix_1)

print('Model_2: 2 layered model')
print(f'Loss: {loss_2}, Accuracy: {accuracy_2}')
print(f'F1 Score: {f1_2}')
print('Confusion Matrix:')
print(conf_matrix_2)

print('Model_3: 3 layered model')
print(f'Loss: {loss_3}, Accuracy: {accuracy_3}')
print(f'F1 Score: {f1_3}')
print('Confusion Matrix:')
print(conf_matrix_3)

print('Model_4: 4 layered model')
print(f'Loss: {loss_4}, Accuracy: {accuracy_4}')
print(f'F1 Score: {f1_4}')
print('Confusion Matrix:')
print(conf_matrix_4)

Model_1: 1 layered model
Loss: 2.0007951259613037, Accuracy: 0.8073540925979614
F1 Score: 0.8007431892323978
Confusion Matrix:
[[964 107]
 [134  46]]
Model_2: 2 layered model
Loss: 2.204350471496582, Accuracy: 0.8065547347068787
F1 Score: 0.8017599415597304
Confusion Matrix:
[[960 111]
 [131  49]]
Model_3: 3 layered model
Loss: 2.7468416690826416, Accuracy: 0.8081534504890442
F1 Score: 0.7996480116520281
Confusion Matrix:
[[968 103]
 [137  43]]
Model_4: 4 layered model
Loss: 1.320464015007019, Accuracy: 0.8129496574401855
F1 Score: 0.8018096299246982
Confusion Matrix:
[[976  95]
 [139  41]]


In [76]:
# Define the feature importance
weights = model_4.get_weights()

feature_importance = np.sum(np.abs(weights[0]), axis=1)

In [77]:
# Define the features
features = df.drop(['EverHadCancer', 'FamilyEverHadCancer2'], axis=1).columns

# Convert feature importance to a DataFrame
feature_importance_df = pd.DataFrame({
    'feature': features,
    'importance': feature_importance
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print(feature_importance_df)


                                              feature  importance
0                                                 Age   13.971249
16                               MedConditions_HighBP   12.945866
9                                      FreqGoProvider   12.213208
8                                           Education   11.527687
53    SunEffectAfter1Hour_Turn darker without sunburn   11.249517
39                          DocTellColorectalTests_No   11.044526
17                       MedConditions_HeartCondition   10.895417
11                                   HeardHPVVaccine2   10.502673
25                                     TimesSunburned   10.422447
48                                 SmokeNow_Some days   10.420902
29                                 BirthGender_Female   10.354181
2                                             AmerInd   10.282412
15                             MedConditions_Diabetes   10.203512
23                                      Sunburned_Rec   10.088268
36        

In [78]:
# Convert DataFrame to a dictionary
FNN_importance = {
    'public': feature_importance_df.to_dict(),  
}
# Create path
save_path = r"C:\Users\kyram\OneDrive\School\FNN_importance.json"

# Confirm the dictionary exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the data to a JSON file
with open(save_path, 'w') as file:
    json.dump(FNN_importance, file)

print(f"Data successfully saved to {save_path}")

Data successfully saved to C:\Users\kyram\OneDrive\School\FNN_importance.json
